<a href="https://colab.research.google.com/github/Shea-Fyffe/transforming-personality-scales/blob/main/tutorials/fine-tuning-transformers-for-big5-inclusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Fine-tuning Transformer Models for Big Five Inclusion
---

This tutorial illistrates how to *fine-tune* (see [Lui et al., 2020](https://doi.org/10.1007/978-981-15-5573-2)) a binary classification **transformer** model to classify whether a scale item is related to the Big Five personality model (i.e., any one of the Big Five factors). With regard to the Big Five, there remains considerable debate about the generality of the model, and whether statistically adjacent traits could be encompassed by it (Costa & McCrae, 1995; Goldberg & Saucier, 1998; John et al., 2008) or lie wholly outside the Big Five (e.g., Ashton et al., 2004; Block, 2010; Paunonen & Jackson, 2000).

To identify if related items are to the Big Five (or their focal classes of interest), we demonstrate how to train a classification model using *2* classes---Big Five items (labeled: `big5_item`) or another type of item (labeled: `other_item`).

To populate the `other_item` class, we collected open-soure items from the following sources:
+ Scales developed to measure traits beyond the Big Five (e.g., CPI, MPQ, JPI)
+ Scales developed to measure non-personality constructs (e.g., CAT-PD, Emotional Intelligence, VIA, Vocational Interests)
+ Synthetically generated items with incorrect grammar

Here, we hope to illustrate a model that will not only flag personality items unrealted to one of the Big Five traits, but will also flag items that may be writted incorrectly or measure a non-personality construct.


---
## Setup
---

Below, we provide information regarding the libraries, functions, and classes used in this tutorial. *Text Blocks* (like this) will serve as informative sign posts. `Code Blocks` which have a black background will actually perform the commands. We recommend adding a *Scratch Code Cell* (**Ctrl+Alt+N**) for running commands interactively. 
<br></br>
**Libraries and Modules**

Colab comes with a large number of Python libraries pre-loaded. However, `Transformers` is not initially available in Colab. The `Transformers` library can be installed by using the code below. More information on the `Transformers` library can be seen [here](https://huggingface.co/transformers/quicktour.html).
<br></br>
**User-Defined Functions and Classes**

Below we provide several classes and functions to help may the process a bit easier. For each function help text is provided and can be printed via `print(fun_name.__doc__)`. For example, to see documentation for the `fine_tune()` function:

```
print(fine_tune.__doc__)

Output:

  Fine-tune transformer model for text classification.

  A wrapper function for fine-tuning a pre-trained transformer
  from the popular transformers library. Abstracts away many of the steps
  involved, such as loading a tokenizer and formatting data.
  
  Arguments
  ---------
  model: a string usually returned from ``get_model()``.
  text: a list of text.
  labels: a list of labels.
  train_args: a dictionary of training arguments.
  multi_label: a boolean specifying whether perform multi-label classification (False by default).
  max_seq_len: a string determining how to pad text sequences ('longest' by default).

  Returns
  -------
  trainer : transformers.Trainer
    a fine-tuned transformer model.
  tokenizer : transformers.tokenizer
    the tokenizer of the fine-tuned model.

```
This provides the arguments that can be modified to customize the fine-tuning process.
<br></br>
**Using a GPU**

To speed things up you can use a *GPU* (*optional*). First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

You can confirm that you have an active GPU by using the following command:
```
# check using a command line interface
!nvidia-smi
```


In [ ]:
#@title Install libraries
## Uncomment command below to install Transformers
! pip install transformers
! pip install sentencepiece

In [ ]:
#@title Import libraries and modules
# load relevant modules from transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer

# data libraries
from torch.utils.data import Dataset # for formatting data before training
import pandas as pd # for importing and exporting data

# util libraries
from scipy.special import softmax
from sklearn.metrics import classification_report
import numpy as np
from google.colab import drive # optional for getting data
from typing import Dict, List, Tuple # for type hinting

import torch
import os
import sys
import datetime
import gc
import warnings
import requests
from io import StringIO

### Functions and Classes



In [ ]:
#@title Data-related functions
# Custom data class
class TextClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

# Import data function
def import_data(path: str, text_col: str = 'text', label_col: str = 'label', enc: str = 'latin1'):
    """
    Import a text data from a csv file.

    A wrapper function around pandas.read_csv. Includes URL support.
    
    Arguments
    ---------
    path: a string indicating a local csv file path or url.
    text_col: a string indicating the name of column in csv containing text ('text' by default).
    label_col: a string indicating the name of column in csv containing text labels ('label' by default).
    enc: a string indicating csv file encoding ('latin1' by default).

    Returns
    -------
    List[str]
      a list of text.
    List[str]
      a list of labels.
    pandas.DataFrame
      the raw data.
    """
    if (path.startswith("http")):
        res = requests.get(path,
                           headers= {'User-Agent': 'Mozilla/5.0',
                                     "X-Requested-With": "XMLHttpRequest"})
        path = StringIO(res.text)
    df = pd.read_csv(path, encoding = enc)
    
    if label_col is None:
      return df[text_col].tolist(), df
    return df[text_col].tolist(), df[label_col].tolist(), df

# Format output data function
def format_output_data(raw_outputs, test_case_ids = None, label_values = None, output_probabilities: bool = True,
                       output_predicted_label: bool = True):
    """
    Format model predictions to DataFrame.

    A helper function that formats classification predictions taken from
    ``transformers.Trainer.predict()`` into various outputs.
    
    Arguments
    ---------
    raw_outputs: a numpy.ndarray of predictions from ``transformers.Trainer.predict()``.
    test_case_ids: a list of test case ids (None by default).
    label_values: a list of *unique ordered* labels (None by default).
    output_probabilities: A boolean specifying whether to convert logit predictions to probabilities (True by default).
    output_predicted_label: A boolean whether to append a 'predicted' column with the most likely label (True by default).

    Returns
    -------
    pandas.DataFrame
      a dataset of predicted values.
    """
    out_df = pd.DataFrame(raw_outputs)

    if output_probabilities:
        out_df = softmax(out_df, axis=1)
    
    if output_predicted_label:
        out_df['predicted'] = np.argmax(out_df, axis=1)
    
    if label_values is not None:
        out_df.columns = label_values
    
    if test_case_ids is not None:
        out_df.insert(0, 'id', test_case_ids)
    return out_df

In [ ]:
#@title Model-related functions
# Custom fine-tuning function
def fine_tune(model, text, labels, train_args, multi_label: bool = False, max_seq_len: str = 'longest', **kwargs):
    """
    Fine-tune transformer model for text classification.
  
    A wrapper function for fine-tuning a pre-trained transformer
    from the popular transformers library. Abstracts away many of the steps
    involved, such as loading a tokenizer and formatting data.
    
    Arguments
    ---------
    model: a string usually returned from ``get_model()``.
    text: a list of text.
    labels: a list of labels.
    train_args: a dictionary of training arguments.
    multi_label: a boolean specifying whether perform multi-label classification (False by default).
    max_seq_len: a string determining how to pad text sequences ('longest' by default).
    kwargs: additional keyword arguments to pass to ``Trainer.__init__``.
  
    Returns
    -------
    trainer : transformers.Trainer
      a fine-tuned transformer model.
    tokenizer : transformers.tokenizer
      the tokenizer of the fine-tuned model.
    """
    _, model_name = get_model(model)
  
    tokenizer = AutoTokenizer.from_pretrained(model_name)
  
    train_labels_indx, lab_to_id, num_labs = map_labels_to_keys(labels)
    
    if max_seq_len == 'longest':
      train_encodings = tokenizer(text, truncation=True, padding=True)
    else:
      train_encodings = tokenizer(text, padding='max_len', max_length=max_seq_len)
  
    train_dataset = TextClassificationDataset(train_encodings, train_labels_indx)
      
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=num_labs, label2id = lab_to_id
        )
    # setting multi_label = True with less than two labels will default to a binary classification
    if multi_label and num_labs > 2:
      model.problem_type = "multi_label_classification"
  
    trainer = Trainer(model=model,
        args = training_args,
        train_dataset = train_dataset,
        **kwargs
      )
   
    trainer.train()
    trainer.tokenizer = tokenizer
    return trainer

# Get model for simple transformers
def get_model(model_type: str) -> Tuple[str, str]:
    """
    Get pre-trained transformer model.
    
    A helper function that looks up pre-trained model given model type. If model
    is *not* found in lookup will ouput string used as input.
    
    Arguments
    ---------
    model_type: a string indicating model type name (e.g., 'bart', 'bert', 'deberta', 'xlnet').
    
    Returns
    -------
    model_name : tuple(str, str)
      a tuple of the model type and specific model name.
      
    See Also
    --------
    See https://huggingface.co/models for the complete repository of usable transformer models      
    """
    model_dict = {
    'albert': ("albert", "albert-xlarge-v2"),
    'bart': ("bart", "facebook/bart-large"),
    'bert': ("bert", "bert-base-cased"),
    'deberta': ("debertav2", "microsoft/deberta-v3-large"),
    'distilbert': ("distilbert", "distilbert-base-cased-distilled-squad"),
    'distilroberta': ("roberta", "cross-encoder/stsb-distilroberta-base"),
    'electra': ("electra", "cross-encoder/ms-marco-electra-base"),
    'roberta': ("roberta", "roberta-large"),
    'xlnet': ("xlnet", "xlnet-large-cased"),
    'xmlroberta': ("xmlroberta", "xlm-roberta-large"),
    }
    # if model is not found will try model_type as model_name
    model_name = model_dict.get(model_type, (model_type, model_type))
    # returns a Tuple  
    return model_name
  
# Compute evaluation metrics
def flag_items(items: List[str], trained_model, prediction_threshold = .50, anchor_label: str = "big5_item", **kwargs):
    """
    Flag items based on some threshold.

    A helper function that flags items based on a classification probability threshold. 
    
    Arguments
    ---------
    items: list of items to flag.
    trained_model: a trained transformer model of class Trainer.
    prediction_threshold: a decimal number between 0.00 and 1.00 to flag items.
    kwargs: additional keyword arguments to pass to ``transformers.Trainer.predict()``.

    Returns
    -------
    pandas.DataFrame
      a dataset of label probabilities and a flag.
    """
    items_encodings = trained_model.tokenizer(items, truncation=True, padding=True)
    items_dataset = TextClassificationDataset(items_encodings)
    
    label_vals = [*get_labels(trained_model).keys()]
    anchor_indx = label_vals.index(anchor_label)
    
    predictions, _, _ = trained_model.predict(items_dataset, **kwargs)
    
    out_df = pd.DataFrame(predictions)
    out_df.columns = label_vals
    
    out_df = softmax(out_df, axis = 1)
    
    out_df["flag"] = out_df.iloc[:, anchor_indx] < prediction_threshold
    
    out_df.insert(0, 'item', items)
    return out_df


In [ ]:
#@title Utility functions
# Map labels to keys
def map_labels_to_keys(labels: List[str], sort_labels: bool = True):
    """
    Map text labels to integers.
    
    This function maps a list of strings to integers.
    
    Arguments
    ---------
    labels: a list of labels
    sort_labels: a boolean specifying if labels should be sorted alphabetically before recoding (True by default)

    Returns
    -------
    List[str]
      a list of labels.
    dict{str : int}
      a dictionary where labels are keys and mapped int are values.
    int
      the number of class labels.
    """
    k = list(dict.fromkeys(labels))
    if sort_labels:
      k.sort()
    labels_to_id = {k[i] : int(i) for i in range(0, len(k))}
    labels_out = []
    for j in labels:
      labels_out.append(labels_to_id[j])
    return labels_out, labels_to_id, len(k)

# Helper to return labels from trained model
def get_labels(trained_model):
    """
    Return list of class labels from a model returned by `Trainer.train()`
    
    Arguments
    ---------
    trained_model: a trained transformer model of class Trainer.

    Returns
    -------
    List[str]
      a list of labels.
    """
    return trained_model.model.config.label2id
     
# Helper to check for GPU device and garbage collect
def get_gpu():
    """
    Check if CUDA compatible GPU is available.

    To manually check if you are able to use a GPU environment in Colab click
    the `Runtime` menu above, then select `Change Runtime Type`, the pick "GPU"
    for the `Hardware Accelerator` dropdown.
    
    Returns
    -------
    int
      number of current CUDA GPU device. If -1, no was found. 
    """
    if torch.cuda.is_available():
      torch.cuda.empty_cache()
      gc.collect()
      return torch.cuda.current_device()
    else:
      return -1
    

---
## Selecting Model and Hyper-Parameters
---
We define our variables for purposes described in our research manuscript. However, we encourage researchers and practitioners to try out alternative models (by manually overriding `transformer_model`). In addition, we wanted to minimize the tuning hyper-parameters during training as the aim of this research is to highlight Transformers in a baseline sense.

In [ ]:
#@title Select pre-trained transformer model
transformer_model = "deberta" #@param ["deberta", "albert", "bert", "bart", "distilbert","distilroberta", "electra", "roberta", "xlnet", "xlmroberta"]

In [ ]:
#@title Define training hyper-parameters

# length to pad items to (~each word is 1.15 sequence units)
SEQ_LEN = 32

# first we can initialized the ClassificationArguments object
training_args = TrainingArguments(
   num_train_epochs = 10,
   learning_rate = 2e-5,
   warmup_ratio = 0.10,
   weight_decay = 0.01,
   per_device_train_batch_size = 32,
   seed = 42,
   evaluation_strategy="no",
   save_strategy="no",
   logging_strategy="epoch",
   output_dir = f"{transformer_model}/outputs",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


---
## Uploading and Importing Data
---

**Uploading Data**

While there are several ways to import data into Colab ([see here](https://colab.research.google.com/notebooks/io.ipynb)), the most intuitive way is to use the project's code repository url:

```
# Assign the online data repository to a url so it does not have to be repeated later
repository_data_path = "https://anonymous.4open.science/api/repo/transforming-personality-scales/file/raw-data/"
```

As an alternative, you can also upload a local `.csv` file. You can do this by:
- Visiting the project url above and clicking the `download file` button (top right in project repository)
- Clicking the ***Files*** pane in Colab (the folder icon on the left in Colab)
- Clicking the ***Upload to session storage*** icon (left-most icon in Colab)
- Selecting the local data file you would like to use (e.g., `.csv`,`.tsv`)

If using this method, the path to the file can be used. To locate the file path using the *Colab File Pane* (folder icon on the left-hand side). Generally, uploaded files will be in the `/content/` directory. Once the file is found, right click the file and select "Copy path." This path can be pasted into the `import_data` function directly or assigned to an object that can be used later.

```
local_file_path = "content/supplemental-item-data.csv"
```
</br>

**Importing Data**

To properly import the training data we must specify the file path, column name containing our items, and column name containing our labels. Then, the `import_data()` returns three objects:

- a list (vector) of items
- a list (vector) of labels
- a copy of our training data

```
# Example using the url
train_text, train_labels, train_raw_data = import_data(repository_data_path + "supplemental-item-data.csv", "text", "label")

# Example using a local file path
train_text, train_labels, train_raw_data = import_data("/" + local_file_path, "text", "label")
```

The code above assigns these to objects names `train_text`, `train_labels` and `raw_data` respectively.

#### Importing the training

We will now import the training data named---`supplemental-item-data.csv`. These data can be found on our [GitHub repo](https://anonymous.4open.science/r/transforming-personality-scales/raw-data/supplemental-item-data.csv) in the directory `raw-data/`.

Unlink our primary example, we will use all of the data collected to train the model. The model can be evaluated during training.

In [ ]:
# Assign the online data repository to a url so it doesn't have to be repeated laterr
repository_data_url = 'https://anonymous.4open.science/api/repo/transforming-personality-scales/file/raw-data/'

# the import_data function will return a list of sentences, a list of labels, and the original dataset
train_text, train_labels, raw_training_data = import_data(repository_data_url + 'supplemental-item-data.csv', "text", "label")

In [ ]:
# here we show the first 20 items in the training set
# ... and their corresponding labels
for x,y in zip(train_text[:20], train_labels[:20]):
    print("Item: %s | Label: %s" %(x, y))

Item: Others perceive that I act before thinking. | Label: big5_item
Item: Being in debt is worrisome to me. | Label: other_item
Item: Others perceive that I believe in the importance of art. | Label: big5_item
Item: Others perceive that I believe people have bad intentions. | Label: big5_item
Item: Others perceive that I cannot handle a lot of information. | Label: big5_item
Item: Others perceive that I my control desires. | Label: big5_item
Item: Others perceive that I disregard rules to get ahead. | Label: big5_item
Item: Others perceive that I do not have original ideas and I am not inventive. | Label: big5_item
Item: Others perceive that I do not joke around. | Label: big5_item
Item: Others perceive that I do not stop to contemplate my life. | Label: big5_item
Item: Others perceive that I enjoy being with people. | Label: big5_item
Item: Others perceive that I get irritated easily. | Label: big5_item
Item: Good manners and cleanliness matter to me. | Label: other_item
Item: Good m

---
## Training the Model
---

To clarify: *fine-tuning* is a specific type of training applied to models that have been pre-trained. This process allows the model to update its parameters to better align with our classification task.

The `fine-tune()` function requires that we define four arguments. We provide a description of each and the (`object`) holding such data:
- The model or type of transformer model (`transformer_model`)
- Text or personality items (`train_text`)
- Text or item class labels (`train_labels`)
- The training hyper parameters (`trainings_args`) 

This results in a function call that looks like:

```
fine_tuned_model, tokenizer = fine_tune(model = transformer_model,
                                        text = train_text,
                                        labels = train_labels,
                                        train_args = training_args)
```

There are several *optional* arguments, such as `max_seq_len` which determines how long text is truncated (discussed below). Additionally, there's the `multi_label` argument&mdash;by setting `multi_label` to `True` i.e., `fine_tune(..., multi_label = True)` one can train a model that will treat items as multi-dimensional, so items may belong to multiple classes at once.


**Tokenizing**

The `fine_tune()` function outputs the fine-tuned model (i.e., `fine_tuned_model`) and add model’s tokenizer to the object (i.e., `fine_tuned_model.tokenizer`). This step ensures both the testing and training items will be tokenized in the same way. Since we will not input the test data to the `fine_tune` function, the model's tokenizer object (i.e., `fine_tuned_model.tokenizer`) will be used right before flagging new items (within the `flag_items()` function).

The `fine_tuned_model.tokenizer()` function has several notable arguments&mdash;`truncation` and `padding`. While truncation is not relevant to our case (because personality items tend to be relatively short text documents), setting `truncation=True` ensures that any document longer than the specified sequence length is truncated. Setting `padding=True`, ensures that any document shorter than the specified sequence length is padded up to that point. Usually transformers default sequence length to 512 tokens; however, it is best practice to set it to a number that is roughly 150% of the words in the longest text document.

**Evaluating the Model**

Since, we are using *all* of our training data, the model will not output evaluation metrics

In [ ]:
# tune the model using the labeled personality items
fine_tuned_model = fine_tune(transformer_model, train_text, train_labels, training_args)

---
## Flagging New Items
---

Since we've trained the model, we can now use the `flag_items()` function to predict if new personality items as well as other types of text documents (e.g., survey responses, social media comments, and performance evaluations) are related to the Big Five or not.

This function takes in new items in addition to a trained model, then outputs prediction probabilities and a `flag` variable representing if the item was *below* the `prediction_threshold`. The `prediction_threshold` is .50 by default.

```
# Create some example items
example_flag_items =  ["I enjoy ice cream", "Swimming is an interesting hobby", "I am the life of the party"]

# flag items example items based on a .50 threshold
flagged_items_1 = flag_items(example_flag_items, fine_tuned_model)

flagged_items_1

# flag items example items based on a .90 threshold
flagged_items_2 = flag_items(example_flag_items, fine_tuned_model, prediction_threshold = .90)

flagged_items_2

```

In [ ]:
#@title Flag New Items
test_items = ["I enjoy ice cream", "Swimming is an interesting hobby", "I make up stories about things that happened that are totally untrue.",
              "I am the life of the party", "Life of party the I am", "Others consider me more of a leader than a follower", 
              "I consider myself more of a leader than a follower", "The capital of canada is ottawa", "zzzz", "I know that the capital of France is Paris"]

# Run the function using the default threshold of .50
flagged_items = flag_items(test_items, fine_tuned_model)

# Print results
flagged_items

***** Running Prediction *****
  Num examples = 10
  Batch size = 8


,item,big5_item,other_item,flag
0,I enjoy ice cream,0.004494,0.995506,True
1,Swimming is an interesting hobby,0.000132,0.999868,True
2,I make up stories about things that happened t...,0.001991,0.998009,True
3,I am the life of the party,0.999744,0.000255,False
4,Life of party the I am,0.001961,0.998039,True
5,Others consider me more of a leader than a fol...,0.999536,0.000464,False
6,I consider myself more of a leader than a foll...,0.997658,0.002342,False
7,The capital of canada is ottawa,0.000803,0.999197,True
8,zzzz,0.001063,0.998937,True
9,I know that the capital of France is Paris,0.000419,0.999581,True


---
### Saving the Model
---
This model can be saved and used to flag non-Big Five items before being sent to the model trained in our [primary example](https://colab.research.google.com/drive/1dNMJ2BuRu2l3JZq1TH0B2Fp6_WEoThXB?usp=sharing).

```
# Save the fine tuned model
fine_tuned_model.save_model("fine-tuned-big5-inclusion-model")
```

In [ ]:
#@title Save fine-tuned model
# Uncomment the line below to save the fine-tuned model for later use
# fine_tuned_model.save_model("fine-tuned-big5-inclusion-model")

---
### Flagging New Examples
---

```
# Load Python libraries
from transformers import AutoModel, AutoTokenizer
from transformers import pipeline

# Import model to flag new items
big5_inc_model = AutoModel.from_pretrained("fine-tuned-big5-inclusion-model")
big5_inc_tokenizer = AutoTokenizer.from_pretrained("fine-tuned-big5-inclusion-model")

# Create classification pipeline
flag_items = pipeline("text-classification", model=big5_inc_model, tokenizer=big5_inc_tokenizer)

# Import or generate items to flag (taken from openpsychometrics.org)
new_items = ["I put family first.",  
             "When other people are arguing, I leave the room.", 
             "I have a bland facial expression when I talk to people.", 
             "Does your heart ever thump in your ears so that you cannot sleep?"]

# Flag Items
results = flag_items(new_items)
```
